# Introduction
[**Sankey diagrams**](https://en.wikipedia.org/wiki/Sankey_diagram) are a powerful tool for visualizing year-on-year changes in a mining block model, specifically illustrating the flow of material between different resource classification classes during resource updates. This allows geologists and mining engineers to track how ore moves between categories like "Inferred," "Indicated," and "Measured" over time. The diagram uses nodes to represent each classification in consecutive years (e.g., "Indicated" in 2023 and "Measured" in 2024). The width of the connecting lines reflects the volume, tonnage, or metal content that shifted between those classifications, highlighting key trends and significant changes in the resource model. This visual representation aids in understanding resource development, supporting better decision-making for mine planning and future operations.

This tutorial will guide you through creating a Sankey diagram in Python using the [`plotly`](https://plotly.com/python/sankey-diagram/) library. We will use a sample dataset to demonstrate how to build a Sankey diagram step by step, from data preparation to visualization. Let's get started!

# Code

## Import Libraries

In [17]:
import numpy as np
import pandas as pd
import plotly as pl

import plotly.graph_objs as go
import plotly.io as pio
pio.renderers.default = "notebook_connected"

## Build Block models

First we generate two blockmodels representing a ore body in 2023 and 2024. 

### 2023 Block model
The 2023 block model consists of a 1000 blocks with randomly generated grades and densities, a *IJK* field and a *mined* flag indicating if a block is mined or not and a *classif* field indicating the resource confidence classification. 

:::{.callout-warning}
Do not use **class** as a variable identifier, it is a reserved keyword in Python.
:::

In [18]:
grade = np.random.normal(7,2,1000) # random grades
density =np.random.normal(3.0,.5,1000) # random density
mined = [0]*1000 # no mined blocks
classif = ['measured']* 200 + \
        ['indicated']* 300 + \
        ['inferred']* 500 # classification
bm_23 = pd.DataFrame({'grade':grade, 'density':density,
                       'mined':mined, 'classif':classif})
bm_23.reset_index(inplace=True)# index to IJK number
bm_23.rename(columns={'index': 'IJK'}, inplace=True)

bm_23.head()

,IJK,grade,density,mined,classif
0,0,7.145957,3.256347,0,measured
1,1,10.510213,2.797827,0,measured
2,2,9.651518,2.789021,0,measured
3,3,6.232049,3.407634,0,measured
4,4,5.496961,3.252509,0,measured


### 2024 Block model
Let us assume that during 2024 new exploration drilling has been done increasing the overall size of the  block model and improving the resource classification in some areas. The grades and densities have also been updated and some blocks were mined out.

In [19]:
grade = np.random.normal(7,2,1050) # random grades
density =np.random.normal(3.0,.5,1050) # random density
mined = [0]*1050 # no mined blocks
classif = ['measured']* 300 + \
        ['indicated']* 600 + \
        ['inferred']* 150 # classification
bm_24 = pd.DataFrame({'grade':grade, 'density':density,
                       'mined':mined, 'classif':classif})
bm_24.reset_index(inplace=True)# index to IJK number
bm_24.rename(columns={'index': 'IJK'}, inplace=True)

# Delete 100 blocks from block model representing
#  losses in 2024 model
bm_24 = bm_24.drop(bm_24.sample(n=10).index)

# Flag the first 50 blocks as mined out
bm_24.loc[0:50, 'mined'] = 1

bm_24.head()

,IJK,grade,density,mined,classif
0,0,4.903689,3.292518,1,measured
1,1,6.378225,3.255630,1,measured
2,2,8.154528,2.618048,1,measured
3,3,5.054425,3.192737,1,measured
4,4,5.671297,2.273916,1,measured


## Prepare the dataframes  

**Workflow:** 

* Calculate the volume, tonnage and metal content. 
* Subset the data frames to include only the required fields.
* Combine the data frames.
* Handle the NA cases.
* Group by the respective *classf* fields and aggregate.
* Calculate the magnitude of the movements between nodes.
* Define node identifiers.

The data frames do not contain the block dimensions, so let's assume that the block \ are 10x10x10 meters in size. The grade is in g/t and the density in t/m3. 

In [20]:
# Calculate volume, tons and metal content.
bm_23['vol'] = 1000
bm_23['tons'] = bm_23['vol']*bm_23['density']
bm_23['metal'] = bm_23['tons']*bm_23['grade']

bm_24['vol'] = 1000 
bm_24['tons'] = bm_24['vol']*bm_24['density']
bm_24['metal'] = bm_24['tons']*bm_24['grade']


To identify the blocks that have been mined after aggregation, the *classif* variable is changed to 'mined'. 

In [21]:
bm_24.loc[bm_24['mined']==1, 'classif'] = 'mined'

In [22]:
# Subset
bm_23_ss = bm_23.drop(columns=(['grade', 'density','mined']))
bm_24_ss = bm_24.drop(columns=(['grade', 'density','mined']))

The two datasets can now be combined using the Pandas [**merge**](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.merge.html) function. It is important to use the *how = 'outer'* argument so that **all** the blocks in both datasets are included. The *merge* function uses *how = 'left'* by default which would exclude the blocks that are **not** present in *bm_23_ss*.

:::{.callout-warning}
The datasets are merged using the *IJK* field as the key. This wil **only work correctly** if both the block models were generated using the **same grid** (block model definition). 
:::

In [23]:
mov = pd.merge(bm_23_ss, bm_24_ss,
                on='IJK',
                how='outer',
                suffixes=('_23', '_24'))
mov.head()

,IJK,classif_23,vol_23,tons_23,metal_23,classif_24,vol_24,tons_24,metal_24
0,0,measured,1000.0,3256.346753,23269.714170,mined,1000.0,3292.517876,16145.484367
1,1,measured,1000.0,2797.827262,29405.759655,mined,1000.0,3255.629776,20765.140825
2,2,measured,1000.0,2789.020585,26918.282244,mined,1000.0,2618.047931,21348.946416
3,3,measured,1000.0,3407.633696,21236.539967,mined,1000.0,3192.737266,16137.451607
4,4,measured,1000.0,3252.509061,17878.914631,mined,1000.0,2273.916107,12896.054578


Blocks in the 2023 model and not in the 2024 model, possibly due to model changes, are assigned a *classif_24* of *'mloss'* (Model Loss). Similarly blocks present in the 2024 model and not in the 2023 model are assigned a *classif_23* of *'mgain'* (Model Gain).  

In [24]:
mov.loc[mov['classif_24'].isna(), 'classif_24'] = 'unclass'
mov.loc[mov['classif_23'].isna(), 'classif_23'] = 'unclass'

Next we group the movements by *classif_23* and *classif_24* and aggregate the volume, tonnage and metal content. Then calculate the magnitude of the movements.

In [25]:
# Accumulate
mov_acc = (mov.groupby(['classif_23','classif_24'])[['vol_23','tons_23','metal_23','vol_24','tons_24','metal_24']]
           .sum()
           .reset_index()
           )

if '_23' not in mov_acc['classif_23'].iloc[0]:
    mov_acc['classif_23'] = mov_acc['classif_23']+'_23'
    mov_acc['classif_24'] = mov_acc['classif_24']+'_24'

# Magnitude of movements
mov_acc['vol'] = mov_acc['vol_24'] 
mov_acc['tons'] = mov_acc['tons_24'] 
mov_acc['metal'] = mov_acc['metal_24'] 

# Address losses
mov_acc.loc[mov_acc['classif_24']=='unclass_24', 'vol'] = mov_acc['vol_23']
mov_acc.loc[mov_acc['classif_24']=='unclass_24', 'tons'] = mov_acc['tons_23']
mov_acc.loc[mov_acc['classif_24']=='unclass_24', 'metal'] = mov_acc['metal_23']


In [26]:
mov_acc

,classif_23,classif_24,vol_23,tons_23,metal_23,vol_24,tons_24,metal_24,vol,tons,metal
0,indicated_23,indicated_24,196000.0,5.756849e+05,4.043772e+06,196000.0,5.882841e+05,4.044052e+06,196000.0,5.882841e+05,4.044052e+06
1,indicated_23,measured_24,100000.0,2.980686e+05,2.037028e+06,100000.0,2.931950e+05,2.159110e+06,100000.0,2.931950e+05,2.159110e+06
2,indicated_23,unclass_24,4000.0,1.389099e+04,8.972613e+04,0.0,0.000000e+00,0.000000e+00,4000.0,1.389099e+04,8.972613e+04
3,inferred_23,indicated_24,394000.0,1.174824e+06,8.200038e+06,394000.0,1.171066e+06,8.445044e+06,394000.0,1.171066e+06,8.445044e+06
4,inferred_23,inferred_24,100000.0,3.008650e+05,2.103936e+06,100000.0,3.018044e+05,2.205205e+06,100000.0,3.018044e+05,2.205205e+06
5,inferred_23,unclass_24,6000.0,1.700072e+04,1.167681e+05,0.0,0.000000e+00,0.000000e+00,6000.0,1.700072e+04,1.167681e+05
6,measured_23,measured_24,149000.0,4.442381e+05,3.030744e+06,149000.0,4.506148e+05,3.171072e+06,149000.0,4.506148e+05,3.171072e+06
7,measured_23,mined_24,51000.0,1.551915e+05,1.159087e+06,51000.0,1.509338e+05,1.061923e+06,51000.0,1.509338e+05,1.061923e+06
8,unclass_23,inferred_24,0.0,0.000000e+00,0.000000e+00,50000.0,1.505624e+05,1.004601e+06,50000.0,1.505624e+05,1.004601e+06


Sankey requires each of the *classif* nodes to have a unique identifier, usually an integer value between0 and n-1, where n is the number of unique nodes. These identifiers are calculated and mapped to the *mov_acc** dataframe.

In [27]:
# Unique node identifiers
node_labels = (list(mov_acc['classif_23'].unique()) + list(mov_acc['classif_24'].unique()))
node_label_map = {k: v for v, k in enumerate(node_labels)}

This yields the node label mapping:

In [28]:
print(node_label_map)

{'indicated_23': 0, 'inferred_23': 1, 'measured_23': 2, 'unclass_23': 3, 'indicated_24': 4, 'measured_24': 5, 'unclass_24': 6, 'inferred_24': 7, 'mined_24': 8}


Finally we map the nodes identifiers to the *classif* as *from* and *to* fields: 

In [29]:
mov_acc['source'] = mov_acc['classif_23'].map(node_label_map)
mov_acc['target'] = mov_acc['classif_24'].map(node_label_map)

In [30]:
mov_acc

,classif_23,classif_24,vol_23,tons_23,metal_23,vol_24,tons_24,metal_24,vol,tons,metal,source,target
0,indicated_23,indicated_24,196000.0,5.756849e+05,4.043772e+06,196000.0,5.882841e+05,4.044052e+06,196000.0,5.882841e+05,4.044052e+06,0,4
1,indicated_23,measured_24,100000.0,2.980686e+05,2.037028e+06,100000.0,2.931950e+05,2.159110e+06,100000.0,2.931950e+05,2.159110e+06,0,5
2,indicated_23,unclass_24,4000.0,1.389099e+04,8.972613e+04,0.0,0.000000e+00,0.000000e+00,4000.0,1.389099e+04,8.972613e+04,0,6
3,inferred_23,indicated_24,394000.0,1.174824e+06,8.200038e+06,394000.0,1.171066e+06,8.445044e+06,394000.0,1.171066e+06,8.445044e+06,1,4
4,inferred_23,inferred_24,100000.0,3.008650e+05,2.103936e+06,100000.0,3.018044e+05,2.205205e+06,100000.0,3.018044e+05,2.205205e+06,1,7
5,inferred_23,unclass_24,6000.0,1.700072e+04,1.167681e+05,0.0,0.000000e+00,0.000000e+00,6000.0,1.700072e+04,1.167681e+05,1,6
6,measured_23,measured_24,149000.0,4.442381e+05,3.030744e+06,149000.0,4.506148e+05,3.171072e+06,149000.0,4.506148e+05,3.171072e+06,2,5
7,measured_23,mined_24,51000.0,1.551915e+05,1.159087e+06,51000.0,1.509338e+05,1.061923e+06,51000.0,1.509338e+05,1.061923e+06,2,8
8,unclass_23,inferred_24,0.0,0.000000e+00,0.000000e+00,50000.0,1.505624e+05,1.004601e+06,50000.0,1.505624e+05,1.004601e+06,3,7


## The Sankey diagram

Finally we can create the plot:

In [31]:
# Sankey plot
fig = go.Figure(data=[go.Sankey(
    arrangement = 'snap',
    node = dict(
      pad = 15,
      thickness = 20,
      line = dict(color = "black", width = 0.5),
      label = list(node_label_map.keys()),
      color = ['#0000ff','#add8e6','#ff0000','#f49460','#0000ff','#ff0000','#f49460','#add8e6','#acd8a6'],
      x = [0.001,0.001,0.001,.5,   1  , 1 ,.5, 1   ,1],
      y = [0.4  ,0.8  ,.1   ,.99 ,.5  ,.15 ,.93, .8 ,.001],
     
    ),
    link = dict(
      source = list(mov_acc['source']),
      target = list(mov_acc['target']),
      value = list(mov_acc['tons']),
     
  ))])

fig.update_layout(title_text="Reconciliation 2023-2024 : Tons",
                  font_size=12,
                  width=600, height=400)
fig.show()

There you go! You have successfully created a Sankey diagram in Python using the `plotly` library.  Please let me know if you have any questions or suggestions.